In [6]:
# Dependencies
import numpy as np
import pandas as pd
import requests
import json

from config import census_key

In [7]:
#API Doc: https://www.census.gov/data/developers/data-sets/acs-5year.html
#Census Variable: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b

In [8]:
# Due to Census' API restraints this code is only valid for 2009-2018. 
# Below list must remain in chronological order
years = [2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]

# Params to be passed into API call, *DO NOT INCLUCE SPACES
census_variables = "B01003_001E" 

'''
CENSUS VARIABLE DESCRIPTIONS:
B01003_001E,	Total population,
'''

'\nCENSUS VARIABLE DESCRIPTIONS:\nB01003_001E,\tTotal population,\n'

In [9]:
#This blank list will hold the yearly API results
results = []

for year in years: 
    
    #API base url is dependent on year
    if year == 2009: #2009 Detailed Tables API Call
        url = f'https://api.census.gov/data/{year}/acs5?key={census_key}&get={census_variables},NAME&for=state:*'      

    elif year == 2010: #2010 Detailed Tables API Call     
        url = f'https://api.census.gov/data/{year}/acs/acs5?key={census_key}&get={census_variables},NAME&for=state:*'
        
    else: #2011-2018 Detailed Tables API Call
        url = f'https://api.census.gov/data/{year}/acs/acs5?get=NAME,{census_variables}&for=state:*&key={census_key}'
        
        
    #Incorporating try/except to catch any year w/o data        
    try:
        print(f'Retrieving {year} data...')
        print(url)
        response = requests.get(url).json()
        results.append(response)
        print(f'{year} data successfully retrieved!')
        print('-------------------')
    
    except:
        print(f'Unable to retrieve {year} data...skipping')
        print('-------------------')
        
print('Data retrieval complete')

Retrieving 2009 data...
https://api.census.gov/data/2009/acs5?key=56fd45ad99936050c5b62b7ea0591503e40b8364&get=B01003_001E,NAME&for=state:*
2009 data successfully retrieved!
-------------------
Retrieving 2010 data...
https://api.census.gov/data/2010/acs/acs5?key=56fd45ad99936050c5b62b7ea0591503e40b8364&get=B01003_001E,NAME&for=state:*
2010 data successfully retrieved!
-------------------
Retrieving 2011 data...
https://api.census.gov/data/2011/acs/acs5?get=NAME,B01003_001E&for=state:*&key=56fd45ad99936050c5b62b7ea0591503e40b8364
2011 data successfully retrieved!
-------------------
Retrieving 2012 data...
https://api.census.gov/data/2012/acs/acs5?get=NAME,B01003_001E&for=state:*&key=56fd45ad99936050c5b62b7ea0591503e40b8364
2012 data successfully retrieved!
-------------------
Retrieving 2013 data...
https://api.census.gov/data/2013/acs/acs5?get=NAME,B01003_001E&for=state:*&key=56fd45ad99936050c5b62b7ea0591503e40b8364
2013 data successfully retrieved!
-------------------
Retrieving 201

In [10]:
#Converting api results into yearly dataframes and consolidating into one large df
for year in years:
    year_index = years.index(year)
    data = results[year_index][1:]
    headers = results[year_index][0]
    
    if year_index == 0:
        first_year = pd.DataFrame(data, columns = headers)
        first_year['Year'] = year
        first_year = first_year[sorted(first_year.columns)]
        census_df = first_year
        
    else:
        headers = results[year_index][0] #setting headers again bc column order is in consistent across year (ie '09 vs '16)
        next_year = pd.DataFrame(data, columns = headers)
        next_year['Year'] = year
        next_year = next_year[sorted(next_year.columns)]
        census_df = pd.concat([census_df, next_year], ignore_index=True)
        
census_df.head()

,B01003_001E,NAME,Year,state
0,683142,Alaska,2009,02
1,4633360,Alabama,2009,01
2,2838143,Arkansas,2009,05
3,6324865,Arizona,2009,04
4,36308527,California,2009,06


In [11]:
#convert api results to interger
census_df = census_df.astype({"B01003_001E" : 'int32'})

In [12]:
#rename variables
census_df = census_df.rename(columns={"B01003_001E" : "Total Population",
                                      'NAME':'State'})

#reorganize columns, dropping state code
census_df = census_df[['State','Year',"Total Population",]]

In [13]:
census_df

,State,Year,Total Population
0,Alaska,2009,683142
1,Alabama,2009,4633360
2,Arkansas,2009,2838143
3,Arizona,2009,6324865
4,California,2009,36308527
...,...,...,...
411,Washington,2016,7073146
412,West Virginia,2016,1846092
413,Wisconsin,2016,5754798
414,Wyoming,2016,583029


In [14]:
census_df.to_csv('../Resources/pop_data.csv', index=False)